In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import numpy as np
import time

# load dataset and labels
X = pickle.load(open('pickle_data/X_train.pickle','rb'))
y = pickle.load(open('pickle_data/y_train.pickle','rb'))

#print(X)
print('X.shape =', X.shape)
print('y.shape =', y.shape)

# normalize pixel data
X = X/255.0

X.shape = (1469, 50, 50, 1)
y.shape = (1469,)


In [3]:
# define hyperparameters
conv_layer = 2
layer_size = 64
dense_layer = 0

In [4]:
# name of model to keep track of changing model architectures
NAME = '{}-conv-{}-nodes-{}-dense-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))
# create a new log of the model to analyze results
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

# define model
model = Sequential()

# 1st convolution
model.add(Conv2D(layer_size, (3,3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional conv layers
for _ in range(conv_layer-1):
    # convolution
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    # Max-Pool to reduce tensor size while retaining distict features
    model.add(MaxPooling2D(pool_size=(2,2)))

# flatten model for fully connected layers
model.add(Flatten())

# add dense layers
for _ in range(dense_layer):
    # fully-connected
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# final fully-connected layer (151 matches number of Pokemon categories)
model.add(Dense(151))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=16, epochs=20, validation_split=0.2, callbacks=[tensorboard])


Train on 1175 samples, validate on 294 samples
Epoch 1/20
1175/1175 [==============================] - 4s 3ms/sample - loss: 1.7648 - accuracy: 0.2817 - val_loss: 1.5258 - val_accuracy: 0.2823
Epoch 2/20
1175/1175 [==============================] - 3s 2ms/sample - loss: 1.3993 - accuracy: 0.4204 - val_loss: 1.3565 - val_accuracy: 0.4592
Epoch 3/20
1175/1175 [==============================] - 3s 2ms/sample - loss: 1.1835 - accuracy: 0.5481 - val_loss: 1.3869 - val_accuracy: 0.4388
Epoch 4/20
1175/1175 [==============================] - 3s 2ms/sample - loss: 1.0322 - accuracy: 0.6136 - val_loss: 1.2120 - val_accuracy: 0.5306
Epoch 5/20
1175/1175 [==============================] - 3s 3ms/sample - loss: 0.8509 - accuracy: 0.6809 - val_loss: 1.1314 - val_accuracy: 0.5986
Epoch 6/20
1175/1175 [==============================] - 3s 2ms/sample - loss: 0.6957 - accuracy: 0.7481 - val_loss: 1.2479 - val_accuracy: 0.5816
Epoch 7/20
1175/1175 [==============================] - 3s 2ms/sample - loss:

In [5]:
# evaluate the model
scores = model.evaluate(X, y, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1469/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

accuracy: 92.51%


In [6]:
# save the model
model.save('models/' + NAME)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/2-conv-64-nodes-0-dense-1573095685/assets
